In [1]:
#fundamental tools for the game environment model
import numpy as np
import multiprocess
import time

# for visualations
import matplotlib.pyplot as plt

In [2]:
from randomchooser import Chooser
from exploresims import simulate_explore

pygame 2.5.1 (SDL 2.28.2, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
ch1 = Chooser(5)
ch2 = Chooser(5, [1,10,10,10,1])

In [4]:
sem = multiprocess.Semaphore(2)
out_lock = multiprocess.Lock()
output_queue = multiprocess.Queue()

In [5]:
lol1 = multiprocess.Process(target=simulate_explore, args=[sem, out_lock, output_queue, 100,100,40, ch1, 15, 1.08])
lol2 = multiprocess.Process(target=simulate_explore, args=[sem, out_lock, output_queue, 100,100,40, ch2, 15, 1.08])

In [6]:
lol1.start()
lol2.start()

In [7]:
lol1.join()
lol2.join()